#TG4: запуск бота на сервере

# Вебхуки и Flask

Long Polling постоянно опрашивает сервера, но если сервер не ответил боту, бот может упасть. Вместо поллинга лучше использовать вебхуки (ссылки, к которым будут обращаться серверы телеграма). 

Для того, чтобы вебхуки заработали, нам нужен полноценный веб-сервер (как вы помните, программа, которая умеет посылать и принимать http(s) запросы). К счастью, мы с вами знакомы с Flask, так что в качестве веб-сервера мы напишем небольшое приложение на фласке, которое будет посредником между нашим ботом и телеграмом.

Начнем с того, что выложим бот на [www.pythonanywhere.com/](https://www.pythonanywhere.com/) - ресурс, где можно бесплатно хостить небольшие приложения. Приложением может быть какой-то сайт или собственно наш бот.

Чтобы выкладывать приложения, нужно зайти на сайт и зарегистрироваться (мы это уже сделали, когда выкладывали сайт). 

   - Обратите внимание, что ваш юзернейм будет адресом вашего приложения. Например, если у вас юзернейм noname, то ваше приложение будет доступно по адресу noname.pythonanywhere.com. 
    
   - После регистрации нужно будет подтвердить свой почтовый адрес.
    
Теперь в наш файл с конфигурациями нужно добавить информацию о нашем хосте. Вот так должен выглядеть `conf.py`:

In [ ]:
TOKEN = "здесь токен"
WEBHOOK_HOST = 'ВАШ ЮЗЕРНЕЙМ.pythonanywhere.com'
WEBHOOK_PORT = '443'  
# телеграм может работать с портами 443, 80, 88 или 8443
# при этом pythonanywhere разрешает использовать только порт 443 - стандартный https порт

А вот так теперь будет выглядеть наш основной код:

In [ ]:
# -*- coding: utf-8 -*-
import flask
import telebot
import conf

WEBHOOK_URL_BASE = "https://{}:{}".format(conf.WEBHOOK_HOST, conf.WEBHOOK_PORT)
WEBHOOK_URL_PATH = "/{}/".format(conf.TOKEN)

bot = telebot.TeleBot(conf.TOKEN, threaded=False)  # бесплатный аккаунт pythonanywhere запрещает работу с несколькими тредами

# удаляем предыдущие вебхуки, если они были
bot.remove_webhook()

# ставим новый вебхук = Слышь, если кто мне напишет, стукни сюда — url
bot.set_webhook(url=WEBHOOK_URL_BASE+WEBHOOK_URL_PATH)

app = flask.Flask(__name__)

# этот обработчик запускает функцию send_welcome, когда пользователь отправляет команды /start или /help
@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
	bot.send_message(message.chat.id, "Здравствуйте! Это бот, который считает длину вашего сообщения.")


@bot.message_handler(func=lambda m: True)  # этот обработчик реагирует все прочие сообщения
def send_len(message):
	bot.send_message(message.chat.id, 'В вашем сообщении {} символов.'.format(len(message.text)))

    
# пустая главная страничка для проверки
@app.route('/', methods=['GET', 'HEAD'])
def index():
    return 'ok'


# обрабатываем вызовы вебхука = функция, которая запускается, когда к нам постучался телеграм 
@app.route(WEBHOOK_URL_PATH, methods=['POST'])
def webhook():
    if flask.request.headers.get('content-type') == 'application/json':
        json_string = flask.request.get_data().decode('utf-8')
        update = telebot.types.Update.de_json(json_string)
        bot.process_new_updates([update])
        return ''
    else:
        flask.abort(403)

К боту мы добавили две строчки: `bot.remove_webhook()` и `bot.set_webhook(...)`. Кроме того мы создали приложение фласка `app = flask.Flask(__name__)`, которое откликается, когда телеграм нам постучится по ссылке `WEBHOOK_URL_PATH`.

Если мы будем выкладывать бота на pythonanywhere, то этого достаточно (остальное pythonanywhere сделает за нас). Если мы будем выкладывать бота куда-то еще (на свой сервер или, к примеру, на Heroku), то в конце кода нужно добавить строчку `app.run(...)` как описано в этом [примере](https://github.com/eternnoir/pyTelegramBotAPI/blob/master/examples/webhook_examples/webhook_flask_echo_bot.py).

# Выкладываем

Выкладывать приложения просто:

**1.** Когда вы заходите на сайт, вы автоматически попадаете в Консоли (__Consoles__). Для начала нам нужно настроить окружение, в котором мы будем работать: поставить питон нужной версии, установить нужные нам модули.

  - Нажимаем __Bash__. Пишем там команды:
   - `mkvirtualenv myvirtualenv --python=/usr/bin/python3.7` - это создаст виртуальное окружение с названием myvirtualenv. На самом деле в вашей папке `/home/ВАШ_ЮЗЕРНЕЙМ/` появится папка `.virtualenvs`, а в ней - папка `myvirtualenv`, в которой будет установлен ваш собственный новый отдельный питон3.7.
   - `pip install flask` - это установит в ваше виртуальное окружение фласк
   - `pip install pyTelegramBotAPI` - это установит в ваше виртуальное окружение модуль pyTelegramBotAPI
   - `which python` - эта команда распечатает путь к вашему питону, он нам понадобится. Там будет что-то такое: `/home/ВАШ_ЮЗЕРНЕЙМ/.virtualenvs/myvirtualenv/bin/python`. 
   
<p style="color:red">** Действия, описанные с этого момента и до конца пункта №1 нужны только для телеграм-ботов! Для обычных flask-приложений вебхуки и ssl вам не понадобятся! Соответственно, в скрипте и конфигах обычного flask-приложения про них тоже ничего быть не должно!**</p>

Чтобы вебхуки заработали, нужно иметь SSL-сертификат (комплект из окрытого и закрытого ключей, ипользующихся в протоколе HTTPS для установки защищенного канала при помощи ассиметричного шифрования), т.к. вебхуки в телеграме работают только по HTTPS. Мы создадим [самоподписанный сертификат](https://ru.wikipedia.org/wiki/%D0%A1%D0%B0%D0%BC%D0%BE%D0%B7%D0%B0%D0%B2%D0%B5%D1%80%D0%B5%D0%BD%D0%BD%D1%8B%D0%B9_%D1%81%D0%B5%D1%80%D1%82%D0%B8%D1%84%D0%B8%D0%BA%D0%B0%D1%82), потому что это быстро и просто, но на самом деле самоподписанные сертификаты не являются надежными, и в продакшене лучше использовать  какой-нибудь публичный центр сертификации, например, [Let's Encrypt](https://help.pythonanywhere.com/pages/LetsEncrypt/). 
    - `openssl genrsa -out webhook_pkey.pem 2048` - генерируем приватный ключ
    - `openssl req -new -x509 -days 3650 -key webhook_pkey.pem -out webhook_cert.pem` - генерируем самоподписанный сертификат
    - После этой команды вам предложат ввести информацию о себе: страна, организация, емейл и т.д. Можно не вводить эти данные, а вместо них ставить точку, но когда дойдете до предложения ввести **Common Name**, следует написать адрес сервера, на котором будет запущен бот - т.е. `ВАШ ЮЗЕРНЕЙМ.pythonanywhere.com`.
    - После этого в вашей домашней директории `/home/ВАШ_ЮЗЕРНЕЙМ/` появится два файла: `webhook_pkey.pem` и `webhook_cert.pem`.

**2.** Чтобы создать приложение на фласке, нужно перейти в Веб (__Web__) - или пройти по ссылке `https://www.pythonanywhere.com/user/ВАШ_ЮЗЕРНЕЙМ/webapps/` - и нажать кнопку __`Add a new web app`__

  - Вам сообщат, что адрес сайта будет связан с вашим юзернеймом. Кликаем Next.
  - Теперь нужно выбрать фреймворк, на котором написан сайт. Кликаем Flask.
  - Теперь нужно выбрать версию питона. Кликаем на ту, которой вы пользовались, когда писали приложение. Например, Python 3.7.
  - Нужно ввести путь к файлу с приложением на сервере. Вам автоматически предлагают что-то вроде `/home/ВАШ_ЮЗЕРНЕЙМ/mysite/flask_app.py`, но я, например, поменяю на  `/home/ЮЗЕРНЕЙМ/mysite/my_app.py`, потому что мой код приложения на фласке находится в файле my_app.py. `mysite` - это название папки, в которой будет жить наш бот.
  - во вкладке Веб (__Web__) нужно найти секцию `Virtualenv` и вставить туда путь к нашему виртуальному окружению, например, что-то такое: `/home/ВАШ_ЮЗЕРНЕЙМ/.virtualenvs/myvirtualenv`.
  - В секции `Code` нужно найти `WSGI configuration file` и кликнуть на этот файл, затем нам нужно заменить его содержимое на такой код (`ВАШ ЮЗЕРНЕЙМ` надо заменить на ваш юзернейм, и не забудьте сохранить - зеленая кнопка SAVE наверху):

In [ ]:
import sys

path = '/home/ВАШ ЮЗЕРНЕЙМ/НАЗВАНИЕ ПАПКИ С БОТОМ'  # например, /home/ЮЗЕРНЕЙМ/mysite
if path not in sys.path:
    sys.path.append(path)

from my_app import app as application  # my_app - это название файла с кодом (фласк + бот)

application.config['DEBUG'] = True
application.config['ssl_context'] = ('/home/ВАШ ЮЗЕРНЕЙМ/webhook_cert.pem', '/home/ВАШ ЮЗЕРНЕЙМ/webhook_pkey.pem')

**3.** Теперь у нас все готово, осталось только загрузить наши файлы с кодом на сервер.  
    
  - Переходим во вкладку Файлы (__Files__), мы попадаем в домашнюю папку (`/home/ВАШ ЮЗЕРНЕЙМ/`). В интерфейсе слева находятся папки (Directories), а справа - файлы. 
  - Находим папку с нашим приложением (`mysite`), переходим в нее. В эту папку нужно загрузить наш файл `my_app.py` с кодом бота и папку `conf.py` с конфигурациями. Интерфейс там довольно понятный: новую папку можно создать слева, загрузить файл - справа.
  - (На самом деле, файлы можно не грузить руками. Вместо этого, можно сначала выложить весь ваш сайт на гитхаб, а потом зайти в Консоли (__Consoles__), и оттуда склонировать ваш реп с сайтом на сервер pythonanywhere в папку `mysite` командой `git clone ССЫЛКА_С_ГИТХАБА mysite`. Но игнорируемые файлы все равно нужно будет загрузить руками!)

Чтобы запустить бота, нужно вернуться во вкладку Веб (__Web__) и перезапустить приложение - кликаем большую зеленую кнопку Reload наверху. 

Готово! Перейдя по ссылке `http://ВАШ_ЮЗЕРНЕЙМ.pythonanywhere.com/`, мы можем проверить, заработал ли фласк: мы должны увидеть там "ок". А можно сразу пойти в телеграм, и написать сообщение своему боту.

Если что-то пошло не так, и вы не видите "ок", то нужно проверить логи: они находятся во вкладке Веб (__Web__) в секции `Log files`. 

## Запуск бота на Heroku

[хендаут](https://github.com/ancatmara/learnpython2018/blob/master/Lessons/Heroku%20%D0%B8%20%D0%BA%D0%BE%D0%BC%D0%B0%D0%BD%D0%B4%D0%BD%D0%B0%D1%8F%20%D1%81%D1%82%D1%80%D0%BE%D0%BA%D0%B0/13a.%20Heroku.md)